In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-25 20:27:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-06-25 20:27:53 (11.2 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("prediction").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Import relevant libraries

from google.colab.patches import cv2_imshow
from google.colab.patches import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from tensorflow.keras.models import load_model
import base64
from imageio import imread
import io
from PIL import Image

In [5]:
# Mount google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
mod_data_file_Path = "/content/drive/MyDrive/Generated_Models/"
mod_data_input_file = "character_recog_model_revision_14.h5"
mod_to_process = mod_data_file_Path + mod_data_input_file
print(mod_to_process)

/content/drive/MyDrive/Generated_Models/character_recog_model_revision_14.h5


In [7]:
# Load trained model
trained_model = load_model(mod_to_process)

In [8]:
# Compile trained model
trained_model.compile(optimizer="adam", loss="binary_crossentropy", metrics = ["accuracy"])

In [9]:
# Create a refence array used for determine model prediction letter

words = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}

In [10]:
# Function to predict letter characters using the trained model
xx = []
def predict_letter_from_word(in_read_image):
    # Read the image using cv2
    image = in_read_image

    # make copy of original image, copy will be used to change colors
    image_copy = image.copy()

    # convert image to RGB using cvtColor
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize image to 400 x 400 size
    image = cv2.resize(image, (400,400))

    # Add blur to image and greyscale (need to greyscale as cv.threshold needs greyscale images)
    image_copy = cv2.GaussianBlur(image_copy, (7,7), 0)
    grey_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)


    # Separate object from background pixels using thresholding
    # https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
    _, img_thresh = cv2.threshold(grey_image, 100, 255, cv2.THRESH_BINARY_INV)

    # Resize and reshape image to fit trained_model requirements
    final_image = cv2.resize(img_thresh, (28,28))
    # xx.append(final_image)
    # print(final_image)
    final_image = np.reshape(final_image, (1, 28, 28, 1))

    # Make prediction using the trained_model
    prediction = words[np.argmax(trained_model.predict(final_image))]

    return(prediction)

In [11]:
def convert_image_to_base64(in_image_file_name):
# Function that takes in image file name reads in mage and convert to a base 64 string
  file_name = (in_image_file_name)
  with open(file_name, "rb") as img_file:
      b64_string = base64.b64encode(img_file.read())

  return(b64_string.decode('utf-8'))

In [12]:
def convert_from_base_64_to_image(in_base_64_image_string):
# Function that takes in base 64 string and converts to image
  image_webP = Image.open((io.BytesIO(base64.b64decode(in_base_64_image_string))))

  image_RGB = image_webP.convert("RGB")
  
  image = np.float32(image_RGB)

  return(image)

In [13]:
def split_image(in_image_to_split, in_no_of_splits , in_section_to_get):
# Function that takes in image and value of what section to extract , calculates start x and exnd x position

  height , width , channels = image_returned.shape

  horizontal_to_extract = width/in_no_of_splits
  
  x_start = int(in_section_to_get * horizontal_to_extract)
  x_end = int(in_section_to_get * horizontal_to_extract + horizontal_to_extract )

  section_part =  in_image_to_split[:, x_start:x_end] 

  return(section_part)

In [14]:


directory_path = '/content/drive/MyDrive/Word_Images/'
directory_listing = os.listdir(directory_path)
for image_name in directory_listing:
    image_file_name = ("/content/drive/MyDrive/Word_Images/" + image_name)
    # COnvert image to base 64 string
    base_64_image_string = convert_image_to_base64(image_file_name)

    # Return an image from the base64 string 
    image_returned = convert_from_base_64_to_image(base_64_image_string)

    # Split image into 3 parts and call prediction function

    no_of_splits = 3
    word_string = ''
    # cv2_imshow(image_returned)
    for split_count in range(no_of_splits):
        image_section = split_image(image_returned , no_of_splits , split_count)
        
        word_string = word_string + predict_letter_from_word(image_section)

    # Output word built from prediction
    print(image_name , word_string)

dog_1.png DOG


In [15]:
# Call mirriam-webster API

url = "https://www.dictionaryapi.com/api/v3/references/collegiate/json/" + word_string + "?key=5a0c68d0-b2d4-4394-8d86-a4d8a68507f2"
# Import the requests library.
import requests
dictionary_response = requests.get(url).json()
dictionary_response


print(dictionary_response)

[{'meta': {'id': 'dog:1', 'uuid': '587c451b-063e-433e-8a25-8371f335eac3', 'sort': '043189000', 'src': 'collegiate', 'section': 'alpha', 'stems': ['dog', 'doglike', 'dogs'], 'offensive': False}, 'hom': 1, 'hwi': {'hw': 'dog', 'prs': [{'mw': 'ˈdȯg', 'sound': {'audio': 'dog00002', 'ref': 'c', 'stat': '1'}}, {'mw': 'ˈdäg', 'sound': {'audio': 'dog00001', 'ref': 'c', 'stat': '1'}}]}, 'fl': 'noun', 'lbs': ['often attributive'], 'def': [{'sseq': [[['sense', {'sn': '1 a', 'dt': [['text', '{bc}{sx|canid||} '], ['vis', [{'t': 'wolves, foxes, and other {wi}dogs{/wi}'}]]], 'sdsense': {'sd': 'especially', 'dt': [['text', '{bc}a highly variable domestic mammal ({it}Canis familiaris{/it}) closely related to the {d_link|gray wolf|gray wolf} '], ['vis', [{'t': "the family's pet {wi}dog{/wi}"}]]]}}], ['sense', {'sn': 'b', 'dt': [['text', '{bc}a male dog '], ['vis', [{'t': '{wi}dogs{/wi} and bitches'}]]], 'sdsense': {'sd': 'also', 'dt': [['text', '{bc}a male usually {d_link|carnivorous|carnivorous} mammal

In [16]:
# From API response , pring the definition

dictionary_response[0]["shortdef"][0]

'canid; especially : a highly variable domestic mammal (Canis familiaris) closely related to the gray wolf'